In [ ]:
!pip install ../input/kerasapplications/keras-team-keras-applications-3b180cb -f ./ --no-index
!pip install ../input/efficientnet/efficientnet-1.1.0/ -f ./ --no-index

import os
import cv2
import pydicom
import pandas as pd
import numpy as np 
import efficientnet.tfkeras
import tensorflow as tf 
import matplotlib.pyplot as plt 
import random
from tqdm.notebook import tqdm 
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from tensorflow_addons.optimizers import RectifiedAdam
from tensorflow.keras import Model
from keras.models import load_model, Model
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.optimizers import Nadam
import seaborn as sns
from PIL import Image

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

from sklearn.cluster import KMeans
from skimage import morphology
from skimage import measure

In [ ]:
def transform_to_hu(slices):
    
    images = np.stack([file.pixel_array for file in slices])
    images = images.astype(np.int16)
    
    preffered_shape = [128, 128]
    if images.shape[1:] !=preffered_shape:
        images_new = np.zeros((images.shape[0],preffered_shape[0],preffered_shape[1]))
        for i in range(images.shape[0]):
                images_new[i]=cv2.resize(images[i], (preffered_shape[0], preffered_shape[1]))
        images=np.copy(images_new)
    
    images[images <= -1000] = 0
    
    # convert to HU
    for n in range(len(slices)):
        
        intercept = slices[n].RescaleIntercept
        slope = slices[n].RescaleSlope
        
        if slope != 1:
            images[n] = images[n].astype(np.float64)*slope
            images[n] = images[n].astype(np.int16)
            
        images[n] += np.int16(intercept)
    return np.array(images, dtype=np.int16)

def z_normalization(image):

    preffered_z_size = 5
    new_image = np.zeros((image.shape[1], image.shape[2], preffered_z_size ))

    if image.shape[0] < preffered_z_size:
        step = int(preffered_z_size/image.shape[0])
        for i in range(image.shape[0]):
            new_image[:,:,i*step] = image[i]
    elif image.shape[0] > preffered_z_size:
        step = int(image.shape[0]/preffered_z_size)
        for i in range(preffered_z_size):
            new_image[:,:,i] = image[i*step]
    else:
        new_image = image
    
    new_image = (new_image)/2**11
    normalization_c = image.shape[0]/preffered_z_size
    return new_image, normalization_c

def get_img(dcm_path):
    
    files = os.listdir(dcm_path)
    file_nums = [np.int(file.split(".")[0]) for file in files]
    sorted_file_nums = np.sort(file_nums)[::-1]
    slices = [pydicom.dcmread(dcm_path + "/" + str(file_num) + ".dcm" ) for file_num in sorted_file_nums]
    
    slices_hu = transform_to_hu(slices)
    slices_norm, c_norm = z_normalization(slices_hu)
    
    return slices_norm, c_norm

In [ ]:
def get_tab(df , norm, num_weeks):
    vector = [(df.Age.values[0]) / 100] 
    
    if df.Sex.values[0] == 'male':
       vector.append(0)
    else:
       vector.append(1)
    
    if df.SmokingStatus.values[0] == 'Never smoked':
        vector.extend([1, 0, 0, 0])
    elif df.SmokingStatus.values[0] == 'Ex-smoker':
        vector.extend([0, 1, 0, 0])
    elif df.SmokingStatus.values[0] == 'Currently smokes':
        vector.extend([0, 0, 1, 0])
    else:
        vector.extend([0, 0, 0, 1])
    
    vector.append(norm/100)
    vector.append(num_weeks/100)
    return np.array(vector)

In [ ]:
C1, C2 = tf.constant(70, dtype='float32'), tf.constant(1000, dtype="float32")

def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)

def qloss(y_true, y_pred):
    # Pinball loss for multiple quantiles
    qs = [0.2, 0.50, 0.8]
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return K.mean(v)

def mloss(_lambda):
    def loss(y_true, y_pred):
        return _lambda * qloss(y_true, y_pred) + (1 - _lambda)*score(y_true, y_pred)
    return loss

In [ ]:
from tensorflow.keras.layers import (
    Dense,Conv3D, Dropout, Activation, Flatten, Input, BatchNormalization, GlobalAveragePooling2D, Add, Conv2D, AveragePooling2D, 
    LeakyReLU, Concatenate 
)
import efficientnet.tfkeras as efn

def get_efficientnet(model, shape):
    models_dict = efn.EfficientNetB5(input_shape=shape,weights=None,include_top=False)
    return models_dict

def build_model(shape=(128,128,5), model_class='b5'):
    inp1 = Input(shape=shape)
    inp2 = Input(shape=(10))
    
    y = Conv2D(filters=3,kernel_size=3, padding='same', strides=(1, 1),data_format='channels_last')(inp1)
   # y = Conv2D(filters=1,kernel_size=3, padding='same', strides=(1, 1),data_format='channels_last')(y)
    
    base = get_efficientnet(model_class, (128,128,3))
    x1 = base(y)
    base.trainable = False
    x1 = BatchNormalization()(x1)
    x1 = GlobalAveragePooling2D()(x1)
    
    x1 = Dropout(0.385)(x1) 
    x1= Dense(20,activation="relu")(x1)
    
    #x2 = tf.keras.layers.GaussianNoise(0.2)(inp2)
    x2 = L.Dense(50, activation="relu")(inp2)
    x2 = L.Dense(30, activation="relu")(x2)

    y = Concatenate()([x1,x2])
    
    p1 = L.Dense(3, activation="linear")(y)
    p2 = L.Dense(3, activation="relu")(y)
    
    preds = L.Lambda(lambda x: x[0] + tf.cumsum(x[1], axis=1))([p1, p2])
     
    
    model = Model([inp1, inp2] , preds)
    #model.load_weights('../input/osic-model-weights/' + 'efficientnetb5-50epochs.h5',by_name=True, skip_mismatch=True)
    
    model.compile(loss=mloss(0.65), optimizer=tf.keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False), metrics=[score])
    return model


model_class = 'b5' 

In [ ]:
test = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv') 


In [ ]:
sun_model =build_model((128,128,5), model_class)
# print('OK')
sun_model.load_weights("../input/retro-nn/modelnn_sun.h5")
# print('OK')
sun_model.summary()

In [ ]:
Root = '../input/osic-pulmonary-fibrosis-progression/'
sub = pd.read_csv(f"{Root}/sample_submission.csv")
nh=len(sub)
num_week=nh//len(test)
fvc = []
confidence = []
names_w = []

for i, pat in enumerate((test.Patient.unique())):
    print(pat)
    
    images_new = np.zeros((num_week, 128, 128, 5))
    vector_new = np.zeros((num_week, 9))
    
    num_weeks = 1
    
    images, c_norm = get_img (Root + 'test/' + pat + '/')
    images = images.astype('float32')
    vector = get_tab (test.loc[test.Patient == pat, :], c_norm, num_weeks)
    base_week = np.resize(np.array(np.arange(num_week)),(num_week,1))
        
    for j in range(num_week):
        images_new[j] = images
        vector_new[j,:-1] = vector
        vector_new[j,-1] = test.Percent.loc[test.Patient == pat]/100
        names_w.append(pat+'_'+str(base_week[j,0]-12))
    
    
    vector_new = np.concatenate((vector_new, base_week), axis = 1)
    vector_new = vector_new.astype('float32')
    print(vector_new)
    buff_predict = sun_model.predict([images_new, vector_new])
    
    confidence.extend(buff_predict[:,2] - buff_predict[:,0])
    fvc.extend(buff_predict[:,1])
    


In [ ]:
# print(confidence)
# print(len(fvc))
# print(names_w)

In [ ]:
# print(PREDICTIONS.shape)
d = {'Patient_Week': names_w, 'FVC': fvc,'Confidence':confidence}
submis=pd.DataFrame(data=d)
# print(submis.head())

In [ ]:
# print(test.head())

for i in range(len(test)):
    submis.loc[submis['Patient_Week']==test.Patient[i]+'_'+str(test.Weeks[i]), 'FVC'] = test.FVC[i]
    submis.loc[submis['Patient_Week']==test.Patient[i]+'_'+str(test.Weeks[i]), 'Confidence'] = 0.1


In [ ]:
df = submis.sort_values(by=['Patient_Week'], ascending=True).reset_index(drop=True)
print(df.head())

In [ ]:
df.to_csv('submission.csv', index=False)